In [1]:
import io
from urllib import request
import csv

response = request.urlopen('https://dq-content.s3.amazonaws.com/251/storm_data.csv')
reader = csv.reader(io.TextIOWrapper(response))

for i, line in enumerate(reader):
    if i < 10:
        print(line)

['ï»¿FID', 'YEAR', 'MONTH', 'DAY', 'AD_TIME', 'BTID', 'NAME', 'LAT', 'LONG', 'WIND_KTS', 'PRESSURE', 'CAT', 'BASIN', 'Shape_Leng']
['2001', '1957', '8', '8', '1800Z', '63', 'NOTNAMED', '22.5', '-140', '50', '0', 'TS', 'Eastern Pacific', '1.140175']
['2002', '1961', '10', '3', '1200Z', '116', 'PAULINE', '22.1', '-140.2', '45', '0', 'TS', 'Eastern Pacific', '1.16619']
['2003', '1962', '8', '29', '0600Z', '124', 'C', '18', '-140', '45', '0', 'TS', 'Eastern Pacific', '2.10238']
['2004', '1967', '7', '14', '0600Z', '168', 'DENISE', '16.6', '-139.5', '45', '0', 'TS', 'Eastern Pacific', '2.12132']
['2005', '1972', '8', '16', '1200Z', '251', 'DIANA', '18.5', '-139.8', '70', '0', 'H1', 'Eastern Pacific', '1.702939']
['2006', '1976', '7', '22', '0000Z', '312', 'DIANA', '18.6', '-139.8', '30', '0', 'TD', 'Eastern Pacific', '1.6']
['2007', '1978', '8', '26', '1200Z', '342', 'KRISTY', '21.4', '-140.2', '45', '0', 'TS', 'Eastern Pacific', '1.30384']
['2008', '1980', '9', '24', '1800Z', '371', 'KAY',

In [2]:
import psycopg2

# Create database
conn = psycopg2.connect(dbname="postgres", user="postgres")
conn.autocommit = True
cursor = conn.cursor()
cursor.execute("CREATE DATABASE hurricanes")
conn.close()

In [6]:
# Create table
conn = psycopg2.connect(dbname="hurricanes", user="postgres")
conn.autocommit = True
cursor = conn.cursor()
cursor.execute('''CREATE TABLE storm_data ( FID integer,
                                            Year integer,
                                            Month integer,
                                            Day integer,
                                            AD_Time integer,
                                            BTID integer,
                                            Name varchar(30),
                                            Lat numeric(4,1),
                                            Long numeric(4,1),
                                            Wind_KTS integer,
                                            Pressure integer,
                                            Cat varchar(10),
                                            Basin varchar(30),
                                            Shape_Leng numeric(7,6)
                                            )''')
conn.close()

In [7]:
conn = psycopg2.connect(dbname="hurricanes", user="postgres")
cursor = conn.cursor()
cursor.execute("select * from information_schema.tables where table_schema = 'public' ")
res = cursor.fetchall()
print(res)
conn.close()

[('hurricanes', 'public', 'storm_data', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None)]


In [10]:
conn = psycopg2.connect(dbname="hurricanes", user="postgres")
conn.autocommit = True
cursor = conn.cursor()

# Create group for analysts
#cursor.execute('CREATE GROUP read_analysts')
cursor.execute('REVOKE all ON storm_data FROM read_analysts')
cursor.execute('GRANT SELECT, INSERT ON storm_data TO read_analysts')


In [11]:
# Create user
cursor.execute("CREATE USER analyst1 WITH PASSWORD 'password'")
cursor.execute('GRANT read_analysts TO analyst1')

conn.close()

In [12]:
# Connect as user
conn = psycopg2.connect(dbname="hurricanes", user="analyst1",password="password")
cursor = conn.cursor()
print(cursor)


<cursor object at 0x000002506FCB9E48; closed: 0>
